### Define Constant

In [1]:
import numpy as np
import paddle.fluid as fluid
c = fluid.layers.fill_constant(shape = [3,4],dtype=np.float32,value=3.14) # Claim constant
print(c)
cpu = fluid.core.CPUPlace() # Load cpu
exe = fluid.Executor(cpu) # Execute,source：either cpu or gpu（NVIDIA, 3G+）
exe.run(fetch_list=[c]) # Fetch constant value，shift + tab: instructions

name: "fill_constant_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 3
      dims: 4
    }
  }
}
persistable: false



[array([[3.14, 3.14, 3.14, 3.14],
        [3.14, 3.14, 3.14, 3.14],
        [3.14, 3.14, 3.14, 3.14]], dtype=float32)]

### Define Variables

#### Method 1

In [1]:
import numpy as np
import paddle.fluid as fluid

# 1、Define Variable
# Don't know the value
v1 = fluid.data(name="v1",shape=[1],dtype='float32') # variable
v2 = fluid.data(name="v2",shape=[1],dtype='float32') # variable

# 2、Define Calculation
add = fluid.layers.elementwise_add(v1,v2) # Addition
mul = fluid.layers.elementwise_mul(v1,v2) # Multiplication


# 3、Define Execution
# Determine the execute in CPU
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)

# 4、Run
x1 = np.array([5]).astype("float32")
y1 = np.array([7]).astype("float32")
outs = exe.run(
        feed={'v1':x1,'v2':y1},
        fetch_list=[add,mul])
print(outs)

[array([12.], dtype=float32), array([35.], dtype=float32)]


#### Method 2

In [1]:
import numpy as np
import paddle.fluid as fluid
# 1、Claim Variables
p1 = fluid.layers.create_parameter(shape = [1],dtype='float32',name = 'p1')
p2 = fluid.layers.create_parameter(shape = [1],dtype='float32',name = 'p2')

# 2、Define Calculation
sub = fluid.layers.elementwise_sub(p1,p2,name = 'sub') # Subtraction

# 3、Define Execution
# Determine the execute in CPU
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)

# 4、Assign value and run
x2 = np.array([15]).astype("float32")
y2 = np.array([7]).astype("float32")
outs = exe.run(
        feed={'p1':x2,'p2':y2}, # feed value
        fetch_list=[sub])

print(outs)

[array([8.], dtype=float32)]


### Addition, average calculation

In [1]:
import numpy as np

In [2]:
np.arange(1,101)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [3]:
from paddle import fluid

# 1、Claim variables
x = fluid.data(name="x",shape=[1],dtype='float32')
y = fluid.data(name="y",shape=[1],dtype='float32')

# 2、Define calculation
result = fluid.layers.elementwise_add(x,y) # Addition

a = fluid.layers.arange(1,101) # From 1 to 100, similar with numpy.arange
b = fluid.layers.cast(a,dtype='float64') # Type conversion: cast
m = fluid.layers.mean(b) # Mean

# 3、Define calculation & CPU
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(feed = {'x':255,'y':3},fetch_list=[result,a,b,m])

[array([258.], dtype=float32),
 array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
         45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
         56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
         67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
         78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
         89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
        100.], dtype=float32),
 array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
         23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
         34.,  35.,  36.,  37.,  38.,  39.,  40., 

In [4]:
a

name: "range_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
    }
  }
}
persistable: false

In [5]:
b

name: "cast_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP64
      dims: -1
    }
    lod_level: 0
  }
}
persistable: false

### Matrix Operations

In [1]:
import paddle.fluid as fluid
import numpy as np

# 1、Claim variables. Define shape, easier for calulation. Claim operation.
a = fluid.layers.data(name='xx', shape=[2, 3], dtype='float32')
b = fluid.layers.data(name='yy', shape=[3, 2], dtype='float32')
out = fluid.layers.matmul(a, b) # Matrix multiplication


# 2、Claim value
a=np.array([[1,2,3],[4,5,6]]).astype('float32')
b=np.array([[1,9],[2,8],[3,7]]).astype('float32')

# 3、Define Execution
# Determine the execute in CPU
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)

# 4、Initialize parameters
#exe.run(fluid.default_startup_program())

# 5、Run
out_y= exe.run(program=fluid.default_main_program(),# Program running in main program
               feed={'xx': a, 'yy': b},# feed value
               fetch_list=[out])
out_y

[array([[ 14.,  46.],
        [ 32., 118.]], dtype=float32)]

In [2]:
display(a,b)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

array([[1., 9.],
       [2., 8.],
       [3., 7.]], dtype=float32)

### Another Matrix Operation: dot

In [1]:
import paddle.fluid as fluid
import numpy as np

# 1、Claim variables
a = fluid.layers.data(name='xx', shape=[3], dtype='float32')
b = fluid.layers.data(name='yy', shape=[3], dtype='float32')

# 2、Claim matrix operation
out = fluid.layers.dot(a, b) 


# 3、Claim value
a=np.array([1,2,3]).astype('float32')
b=np.array([1,9,2]).astype('float32')


# 4、Define Execution
place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)

# 5、Run
out_y= exe.run(feed={'xx': a, 'yy': b},# feed
               fetch_list=[out]) # fetch
out_y

[array([25.], dtype=float32)]

### Fetch parameters

In [1]:
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr #Parameter attribute
import numpy as np

# 1、Claim variable
x = fluid.layers.data(name='x', shape=[None,5], dtype='float32')

# 2、Define parameters and name them
param_w = ParamAttr(name='target_fc') # Parameter: weight
bias_b = ParamAttr(name='target_fc_b') # Parameter: bias

# 3、Fully connected layer matrix operation
# fc layer,automatically set a weight for x
fc1 = fluid.layers.fc(input=x, # input data
                      size=10, # output size
                      act=None, # activation fcn
                      param_attr=param_w, # weight
                      bias_attr=bias_b) # bias

# 4、Define Execution
place = fluid.CPUPlace()
exe = fluid.Executor(place)

# 5、Initialize，Fetch parameters
exe.run(fluid.default_startup_program())
w = fluid.global_scope().find_var("target_fc").get_tensor()
b = fluid.global_scope().find_var("target_fc_b").get_tensor()
w = np.array(w)
b = np.array(b)
print (w.shape,b.shape)
print(w,b)

a = np.asarray([1,2,3,4,6]).astype(np.float32).reshape(1,5)
# 6、Run
exe.run(feed = {'x':a},fetch_list = [fc1])

(5, 10) (10,)
[[-0.31438428 -0.50595033  0.17520362  0.06058151 -0.14575395 -0.25443906
   0.25948232  0.3222621   0.03834236  0.2610249 ]
 [ 0.15416425  0.19081819 -0.09767336 -0.4684856  -0.18790025  0.54540664
  -0.53641254 -0.4412768   0.22943074  0.55534583]
 [-0.16143468  0.5039485   0.59373933  0.03477418  0.0455507   0.36226302
  -0.5978517   0.04699957 -0.5348689  -0.55646163]
 [-0.57261074  0.34489322 -0.41904807  0.5876327  -0.01207119  0.43487543
  -0.59538144  0.3857563   0.08492589 -0.11790186]
 [-0.40498537  0.1516527   0.38702458  0.595934   -0.27476904  0.48081988
  -0.19739449  0.14494753  0.53857106 -0.35096446]] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


[array([[-5.210715 ,  3.6770205,  2.40703  ,  5.1540675, -2.0818014,
          6.547584 , -6.1727905,  1.9934177,  2.463727 , -2.8750625]],
       dtype=float32)]

In [2]:
# numpy matrix operation
a.dot(w)

array([[-5.210715 ,  3.6770205,  2.40703  ,  5.1540675, -2.0818014,
         6.547584 , -6.1727905,  1.9934177,  2.463727 , -2.8750625]],
      dtype=float32)

### program


In [1]:
import numpy as np
import paddle.fluid as fluid

# 1、Claim variables
x = fluid.data(name='x',shape=[None, 13], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')


# Program
# 2.1、FC matrix opeartion
y_predict = fluid.layers.fc(input=x, size=1, act=None)
# 2.2、Square error
cost = fluid.layers.square_error_cost(input=y_predict, label=y)

# 3、Define Execution
cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())

# 4、Feed and fetch
a = np.random.randint(0,10,size =[1,13]).astype(np.float32)
b = np.random.randint(0,10,size = 1).astype(np.float32)
y_,cost_ = exe.run(feed = {'x':a,
                           'y':b},
                   fetch_list=[y_predict,cost])
display(y_,cost_)

array([[-4.5333385]], dtype=float32)

array([[90.884544]], dtype=float32)

$$cost = \sum\limits_{i = 1}^n(\hat{y_i} - y_i)^2/n$$

In [2]:
# Square error
((y_ - b)**2).mean()

90.884544